In [2]:
# import the training dataset

import pandas as pd

dataset = pd.read_csv('/content/drive/My Drive/NLP/Assignment 2/data/traindata.csv', sep='\t', header= None)
dataset = dataset.loc[:, [0, 1, 2,3, 4]]
dataset = dataset.rename(index=str, columns={ 0: "sentiment", 1: "aspect_category",2: "word", 3:"position", 4: "review"})

dataset.head(5)

,sentiment,aspect_category,word,position,review
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very..."


In [0]:
from sklearn.preprocessing import LabelEncoder

In [4]:
# doing laber encoding for the sentiment

label_encoder = LabelEncoder()
dataset['integer_sentiment'] = label_encoder.fit_transform(dataset.sentiment)

dataset.integer_sentiment.unique()

array([2, 0, 1])

In [5]:
dataset.head(5)

,sentiment,aspect_category,word,position,review,integer_sentiment
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...,2
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...,2
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...,2
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.,0
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very...",2


# Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [6]:
!pip install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
     |████████████████████████████████| 3.7MB 37.8MB/s 
     |████████████████████████████████| 1.0MB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=db202a3847baa56d566a35c776e47f0a6e3bb5adbafc9f2077072aee8e374a48
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


### Imorting the required packages

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from sklearn.svm import SVC
warnings.filterwarnings('ignore')

# Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

Note - DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance

In [34]:

## Loading the BERT model
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-large-uncased')

# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Note:

Instead of downloading the BERT model using the above cell, you can [download](https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz) the model manually and then load it usinng the below code.

In [1]:
'''
The below code can be used

from transformers import BertModel, BertTokenizer
model = BertModel.from_pretrained('/content/drive/My Drive/NLP/') # this is the path to the model 
tokenizer=BertTokenizer.from_pretrained('/content/drive/My Drive/NLP/vocab.txt') # this is the txt file to create vocabulary

'''

"\nThe below code can be used\n\nfrom transformers import BertModel, BertTokenizer\nmodel = BertModel.from_pretrained('/content/drive/My Drive/NLP/') # this is the path to the model \ntokenizer=BertTokenizer.from_pretrained('/content/drive/My Drive/NLP/vocab.txt') # this is the txt file to create vocabulary\n\n"

# Preparing the sentences 
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

## 1. Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [0]:
tokenized = dataset['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# Padding  

After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [37]:
# Our dataset is now in the padded variable, we can view its dimensions below:

np.array(padded).shape

(1503, 92)

# Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [38]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(1503, 92)

# TRANSFER LEARNING  
The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
# We'll save those in the features variable, as they'll serve as the features to our logitics regression model or an SVM

features = last_hidden_states[0][:,0,:].numpy()

In [0]:
# The labels indicating which sentence is positive and negative now go into the labels variable

labels = dataset['integer_sentiment']

# Now we have the features and the labels from the train data.  
let chech their shape/ header etc. 

In [42]:
print(features.shape)
print(labels.shape)
print(type(features))
print(type(labels))

(1503, 1024)
(1503,)
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


# Creating the features from the test data (dev data)

In [43]:
test_dataset = pd.read_csv('/content/drive/My Drive/NLP/Assignment 2/data/devdata.csv', sep='\t', header= None)
# test_dataset = test_dataset.loc[:, [1, 2,3, 4]]
test_dataset = test_dataset.rename(index=str, columns={ 0: "sentiment", 1: "aspect_category",2: "word", 3:"position", 4: "review"})

test_dataset.head(5)

,sentiment,aspect_category,word,position,review
0,positive,LOCATION#GENERAL,neighborhood,54:66,"great food, great wine list, great service in ..."
1,negative,RESTAURANT#GENERAL,place,15:20,I thought this place was totally overrated.
2,positive,FOOD#QUALITY,Fish,0:4,Fish is so very fresh.
3,negative,SERVICE#GENERAL,manager,19:26,"I showed it to the manager, and he smilingly a..."
4,neutral,DRINKS#QUALITY,margaritas,63:73,"The food we ordered was excellent, although I ..."


In [0]:
# doing laber encoding for the sentiment

label_encoder = LabelEncoder()
test_dataset['integer_sentiment'] = label_encoder.fit_transform(test_dataset.sentiment)

# repeating the same  preporcessing steps as that of the train data set

In [0]:
# 1. Tokenization

tokenized_test = test_dataset['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [0]:
# 2. Padding 

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

In [47]:
# Our dataset is now in the padded variable, we can view its dimensions below:

np.array(padded_test).shape

(376, 92)

In [48]:
# 3. Masking

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

(376, 92)

In [0]:
# transfer learning

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states_test = model(input_ids_test, attention_mask=attention_mask_test)

In [0]:
# We'll save those in the features variable, as they'll serve as the features to our logitics regression model or an SVM

features_test = last_hidden_states_test[0][:,0,:].numpy()

In [0]:
# The labels indicating which sentence is positive and negative now go into the labels variable

labels_test = test_dataset['integer_sentiment']

# Now we can use this output in various ML models

These models will take in the result of DistilBERT’s processing, and classify the sentence 

# 1. LINEAR REGRESSION

In [52]:
lr_clf = LogisticRegression()
lr_clf.fit(features,labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
lr_clf.score(features_test, labels_test)

0.8191489361702128

# 2. SVM

In [54]:
clf = SVC(gamma='auto')
clf.fit(features,labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [55]:
clf.score(features_test, labels_test)

0.8404255319148937

# 3. Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
clf = RandomForestClassifier(n_estimators = 200, max_depth=200, random_state=0)

clf.fit(features,labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [58]:
clf.score(features_test, labels_test)

0.7872340425531915

# Hence, the best results were obtained by using BERT model with SVM.
